## Using Python and Pandas Dataframes to Perform a MERGE (UPSERT) Operation
When managing a data warehouse stored in a relational database management system (RDBMS) like Oracle, SQL Server, PostgreSQL or MySQL, a common requirement is to perform incremental updates of the dimension tables. When managing slowly-changing dimension (SCD) Type 1 changes, it is necessary to INSERT any new rows identified in the source database table(s) while UPDATING any existing rows where changes (updates) have been detected in the source database table(s).

In this lab you will be refreshing the **Northwind_DW2** dimensional database you created earlier in Lab 3 using one of the most popular methods: the intrinsic capacity of the **Pandas Dataframes** to **Insert** new data and **Update** existing data.

### 1.0. Prerequisites:
This notebook uses the SqlAlchemy database connectivity library to connect to MySQL databases; therefore, if you have not already done so, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install sqlalchemy`

#### 1.1. Import the Necessary Libraries

In [ ]:
import os
import numpy
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text

print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")

#### 1.2. Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

In [ ]:
host_name = "localhost"  #"jtupitz-mysql.mysql.database.azure.com"
port = "3306"
user_id = "jtupitza"
pwd = "Passw0rd123"

src_dbname = "northwind"
dst_dbname = "northwind_dw2"

#### 1.3. Define Functions for Getting Data From and Setting Data Into Databases

In [ ]:
def get_mysql_connection(user_id, pwd, host_name, db_name):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    return connection
    

def get_dataframe(db_connection, sql_query):
    dframe = pd.read_sql(text(sql_query), db_connection);
    db_connection.close()
    
    return dframe


def set_dataframe(db_connection, df, table_name, pk_column, db_operation):
    try:
        if db_operation in ['insert', 'update']:
            if db_operation.lower() == "insert":
                df.to_sql(table_name, con=db_connection, index=False, if_exists='replace')
                db_connection.execute(text(f"ALTER TABLE {table_name} ADD {pk_column} INT AUTO_INCREMENT PRIMARY KEY FIRST;"))
                    
            elif db_operation.lower() == "update":
                df.to_sql(table_name, con=db_connection, index=False, if_exists='append')

        else:
            print("The value supplied to the 'db_operation' parameter must be either 'insert' or 'update'.")
            
    except Exception as e:
        return f"An error occured: {e}"
        
    db_connection.close()


def upsert_dataframes(source_df, source_key, target_df, target_key, columns):
    '''Perform a MERGE (Upsert) operation using Pandas Dataframes.'''
    # 1. Set the index for both DataFrames to their common business-key
    target_df.set_index(source_key, inplace=True)
    source_df.set_index(source_key, inplace=True)

    # 2. Update Existing Rows in the Target with New Values from the Source
    target_df.update(source_df)  # Update rows with matching indices
    
    # 3. Add New Rows from the Source to the Target 
    df_upserted = target_df.combine_first(source_df).sort_index()

    # 4. Set the index back to its default (not a column in the dataframe)
    df_upserted.reset_index(level=None, inplace=True)

    # 5. Drop the in-memory primary key of the Target.
    df_upserted.drop(target_key, axis=1, inplace=True)
    
    return df_upserted[columns]

### 2.0. Use Python and Pandas Dataframes to Apply Upsert Logic
This approach works well for small to mid-sized tables as data must first be marshalled from both the source and destination (target) database management servers to the Python client, and the **upserted** result must then marshalled back to the target database management server to persist the changes.

#### 2.1. Execute Some Changes to the Source Database **northwind**
Before proceding any further, you **must** first execute the SQL Script named <a href="https://github.com/JTupitza-UVA/DS-2002/blob/main/02-Python/data/Lab_03b_Modify_Data.sql"><b>Lab_03b_Modify_Data.sql</b></a> located in the **/02-Python/data** folder. This script **inserts** new rows, and **updates** existing rows in the dimension tables: dim_customers, dim_employees, dim_products, and dim_shippers.

#### 2.2. Extract Data from the Transaction Processing (source) Database Tables
First, we will fetch data for each dimension table (e.g., customers, employees, products, shippers) from the original **northwind** (source) database using the **get_dataframe()** function. Any changes (i.e., newly added rows, and updates made to existing rows) can be detected.

For optimal performance these changes can be incrementally extracted either from **Change Data Capture (CDC)** tables when the source RDBMS supports that feature, or by filtering rows using either a **last_modified** datetime column or an **is_current** boolean (a.k.a., flag) column that can be included in the schema design of those source tables.

In [ ]:
# 1. Get a connection object to the source server
src_connection = get_mysql_connection(user_id, pwd, host_name, src_dbname)

# 2. Extract a Pandas DataFrame containing the most recent version of the source data.
sql_customers = "SELECT * FROM northwind.customers;"
df_customers = get_dataframe(src_connection, sql_customers)

# 3. Enumerate the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['email_address','home_phone','mobile_phone','web_page','notes','attachments']
df_customers.drop(drop_cols, axis=1, inplace=True)

# 4. Rename the 'id' column to reflect the entity represented by the table
df_customers.rename(columns={"id" : "customer_id"}, inplace=True)
df_customers.tail(4)

In [ ]:
#TODO: Extract a Pandas DataFrame containing the most recent version of the source data from the "employees" table in the Northwind database.

#TODO: Drop the following columns from the 'df_employees' Dataframe: mobile_phone, notes, and attachments.

#TODO: Rename the 'id' column to 'employee_id'

df_employees.tail(4)

In [ ]:
#TODO: Extract a Pandas DataFrame containing the most recent version of the source data from the "products" table in the Northwind database.

#TODO: Drop the following columns from the 'df_products' Dataframe: supplier_ids, description, and attachments.

#TODO: Rename the 'id' column to 'product_id'

df_products.tail(4)

In [ ]:
#TODO: Extract a Pandas DataFrame containing the most recent version of the source data from the "shippers" table in the Northwind database.

#TODO: Drop the following columns from the 'df_shippers' Dataframe:
     # last_name, first_name, email_address, job_title, business_phone, home_phone, mobile_phone, fax_number, web_page, notes, and attachments.

#TODO: Rename the 'id' column to 'shipper_id'

df_shippers.tail(4)

##### 1.4. Extract Data from the Data Warehouse (target) Tables.
The next step is to retrieve the current state of the **target** data warehouse. This can be used to detect the delta (difference) between the **source** (northwind) and destination (northwind_dw2) database's tables.

In [ ]:
src_connection = get_mysql_connection(user_id, pwd, host_name, src_dbname)

sql_dim_customers = "SELECT * FROM northwind_dw2.dim_customers;"
df_dim_customers = get_dataframe(src_connection, sql_dim_customers)
df_dim_customers.tail(4)

In [ ]:
#TODO: Fetch data from the 'dim_employees' table in the northwind_dw2 data warehouse database.

In [ ]:
#TODO: Fetch data from the 'dim_products' table in the northwind_dw2 data warehouse database.

In [ ]:
#TODO: Fetch data from the 'dim_shippers' table in the northwind_dw2 data warehouse database.

##### 1.3. Apply Upsert Logic
Calling the **upsert_dataframes()** function returns a dataframe containing the product of a **Merge** being applied to the **target** dimension tables.

In [ ]:
# 1). List the preferred column order for the final Customers dimension table.
ordered_cols=['customer_id', 'company', 'last_name', 'first_name', 'job_title', 'business_phone'
              , 'fax_number', 'address', 'city', 'state_province', 'zip_postal_code', 'country_region']

# 2). Call the upsert_dataframes function passing the source dataframe, business_key, target dataframe, surrogate primary key, and ordered columns list
df_dim_customers = upsert_dataframes(df_customers, "customer_id", df_dim_customers, "customer_key", ordered_cols)
df_dim_customers.tail(4)

In [ ]:
# 1). List the preferred column order for the final Employees dimension table.

# 2). Call the upsert_dataframes function passing the source dataframe, business_key, target dataframe, surrogate primary key, and ordered columns list

df_dim_employees.tail(4)

In [ ]:
# 1). List the preferred column order for the final Products dimension table.

# 2). Call the upsert_dataframes function passing the source dataframe, business_key, target dataframe, surrogate primary key, and ordered columns list

df_dim_products.tail(4)

In [ ]:
# 1). List the preferred column order for the final Shippersyees dimension table.

# 2). Call the upsert_dataframes function passing the source dataframe, business_key, target dataframe, surrogate primary key, and ordered columns list

df_dim_shippers.tail(4)

##### 1.4. Load the newly UPSERTED dataframes into the Data Warehouse
Calling the **set_dataframe()** function uses the **Upserted** dataframes to created updated versions of the dimension tables in the **northwind_dw2** (target) data warehouse.

In [ ]:
# 1). Call the get_mysql_connection() function to create a connection to the target (northwind_dw2) database
dst_connection = get_mysql_connection(user_id, pwd, host_name, dst_dbname)

# 2). Call the set_dataframe function passing the connection object, upserted dataframe, and the names of the dimension table, primary key, and "insert"
set_dataframe(dst_connection, df_dim_customers, "dim_customers", "customer_key", "insert")

In [ ]:
# 1). Call the get_mysql_connection() function to create a connection to the target (northwind_dw2) database
dst_connection = get_mysql_connection(user_id, pwd, host_name, dst_dbname)

# 2). Call the set_dataframe function passing the connection object, upserted dataframe, and the names of the dimension table, primary key, and "insert"

In [ ]:
# 1). Call the get_mysql_connection() function to create a connection to the target (northwind_dw2) database
dst_connection = get_mysql_connection(user_id, pwd, host_name, dst_dbname)

# 2). Call the set_dataframe function passing the connection object, upserted dataframe, and the names of the dimension table, primary key, and "insert"

In [ ]:
# 1). Call the get_mysql_connection() function to create a connection to the target (northwind_dw2) database
dst_connection = get_mysql_connection(user_id, pwd, host_name, dst_dbname)

# 2). Call the set_dataframe function passing the connection object, upserted dataframe, and the names of the dimension table, primary key, and "insert"

##### 1.5. Verify (Unit Test) Upsert Results
Finally, we can re-query the state of each dimension table to verify that the new changes now exist in each dimension table.

In [ ]:
# 1). Call the get_mysql_connection() function to create a connection to the target (northwind_dw2) database
src_connection = get_mysql_connection(user_id, pwd, host_name, src_dbname)

# 2). Call the get_dataframe function to the entire contents of the 'dim_customers' table.
sql_dim_customers = "SELECT * FROM northwind_dw2.dim_customers;"
df_dim_customers = get_dataframe(src_connection, sql_dim_customers)
df_dim_customers.tail(4)

In [ ]:
#TODO: Fetch data from the 'dim_employees' table in the northwind_dw2 data warehouse database.

In [ ]:
#TODO: Fetch data from the 'dim_products' table in the northwind_dw2 data warehouse database.

In [ ]:
#TODO: Fetch data from the 'dim_shippers' table in the northwind_dw2 data warehouse database.